# Introduction to LumiSpy

> **Luminescence Spectroscopy Data Analysis in Python Using [HyperSpy](https://hyperspy.org)**

Tutorial for the **2023 Sfµ Atelier: HyperSpy** at the Sfµ colloque in Rouen

> Rouen, July 4, 2023

## Introduction to LumiSpy

### What is LumiSpy?

- **HyperSpy** extension for luminescence spectroscopy data
- Provides the framework to work with photoluminescence (PL), cathodoluminescence (CL), electroluminescence (EL) and Raman spectroscopy
- Adds **specific functions**
- Work in progress ... open for other contributors

### Resources: Getting started and obtaining help
- User guide: https://lumispy.org
- Tutorial notebooks: https://github.com/LumiSpy/lumispy-demos
- HyperSpy resources: [Website](https://hyperspy.org), [User guide](https://hyperspy.org/hyperspy-doc/current/), [Tutorial notebooks](https://github.com/hyperspy/hyperspy-demos), [Gitter chat](https://gitter.im/hyperspy/hyperspy)

## Import packages

We import the public functions (api = application programming interface) of `HyperSpy`.

Object-oriented functionalities of `LumiSpy` are directly available if the package is installed, but we can separately load it to access extra utilities.

In [ ]:
# silence some "WARNINGS" (only distracting at this stage)
import warnings; warnings.simplefilter('ignore')

# Use '%matplotlib widget' in JupterLab and '%matplotlib notebook' in JupyterNotebook for interactive inline functionality (e.g. on binder)
# For pop-up window plots on your local computer, use '%matplotlib tk' or '%matplotlib qt' instead
%matplotlib qt

import hyperspy.api as hs
import lumispy as lum

LumiSpy provides a number of signal classes for luminescence spectroscopy, in particular
- Luminescence: wavelength/energy/wavenumber axis
  -  CL, PL, EL, ...
- Transient: time axis
- TransientSpec: 2D signal (e.g. streak camera image) with wavelength/... and time axes

We can check the **available signal types**:

In [ ]:
hs.print_known_signal_types()

## Loading files

[HyperSpy](https://hyperspy.org)/[RosettaSciIO](https://hyperspy.org/rosettasciio) support a wide range of spectroscopy related **data file types**:

- Gatan `.dm3/.dm4`
- Attolight `.sur`
- Delmic `.hdf5` (Currently a pull request to RosettaSciIO)
- Hamamatsu streak camera images in `.tif` format (Reading the itex `.img` format is currently a pull request to RosettaSciIO)
- ... many microscopy specific formats

Further spectroscopy-related formats are available through **RosettaSciIO** (fully integrated from HyperSpy v2.0):

- Horiba JobinYvon XML `.xml`
- TriVista XML `.tvf`
- Renishaw wire format `.wdf`

We will one spectral map  in the `dm4` format that we will use during the demo:

*We assume the file location as in the demo repository, if you downloaded the notebook and the data files individually, you might need to adapt the path.*

*You can also leave the path empty. A pop-up window will appear to select.*

In [ ]:
cl2 = hs.load('../demo-files/load_from_GatanFiles/asymmetric-peak_map.dm4')

The `signal_type` file is directly set to `CLSpectrum` if LumiSpy is installed. The spectral map contains two navigation and one signal dimension:

In [ ]:
cl2

The `metadata` contains CL specific information from the vendor format in the `Detector`, `Spectrometer` and `Spectrum_image` nodes:

In [ ]:
cl2.metadata

## Plot / Explore

We can easily plot and explore the hyperspectral data (drag the marker in the *navigation* window to change the displayed spectrum):

In [ ]:
cl2.plot()

## Correction of spectral defects

The dataset is unprocessed, so we will first use some basic functions for artefact correction:

### Remove background (interactive)

HyperSpy has an interactive tool for **background removal** that supports various functions, let's start by removing a **simple offset**:
1. Select a region to be used to determine the background (lowest signal intensity): On the signal plot click, drag and release
2. Select the background type *Offset* (can also be set using the argument `background_type="Offset"`)
3. You can still move the region or its boundaries with the mouse and inspect the different spectra using the navigator to make sure the region is right
4. Press `Apply`

In [ ]:
cl2.remove_background()

### Remove last pixels from the spectrum

The signal beyond 800 nm goes to negative values, so lets remove the last three pixels from every spectrum (using signal indexation) and replace the original signal.

*NOTE: Indexation operates on pixel in the signal dimension if the given number is an integer and on the calibrated (wavelength axis) if a float value is used as index.*

In [ ]:
cl2 = cl2.isig[:-3]

In [ ]:
cl2.plot()

### Remove spikes (interactive)

There is also a tool for interactive removal of cosmic rays (pixels with sharp spikes), see `Help` for instructions.

In brief:
- Inspect the derivative histogram
- Set a sensible threshold to catch the outliers in the histogram (8 is a sensible threshold for this dataset)
- Iterate through `Find next` / `Remove spike` to continue for wrong identifications / remove identified spikes
- `Close` when finished

*NOTE: The interactive version does not work well with inline plotting (e.g. on binder). You can also do an automatic best guess spikes removal by passing `interactive=False`.*

In [ ]:
cl2.spikes_removal_tool()

### Data smoothing

The current dataset is quite noisy. As the peak is broad in comparison with the spectral resolution, one way to improve that is by **rebinning** the data along the signal axis:

In [ ]:
cl2 = cl2.rebin(scale=[1,1,2])
cl2.plot()

Additionally, HyperSpy provides three different functions for **data smoothing**:

- `smooth_lowess` (lowess smoothing)
- `smooth_savitzky_golay` (Savitzky Golay filter)
- `smooth_tv` (total variation data smoothing)

These functions can be run interactively to choose the right parameters, but the parameters can also be passed to the function. You can play with the parameters and get a live preview, and hit `Apply` when you are happy with the smoothed curve.

*As we want to use the non-smoothed data later for fitting the data, we first make a copy of the dataset.*

In [ ]:
cl2a = cl2.deepcopy()
cl2.smooth_lowess(number_of_iterations = 2)

## Peak identification / Centroid / Peak width

In particular for asymmetric peaks, fitting might not always be the best way to determine peak characteristics (though asymmetric functions, such as the skew normal distribution are provided). Therefore HyperSpy provides a number of additional routines.

Peaks can be identified and characterized using the **peak finder** routine `find_peaks1D_ohaver` that is based on the downwards zero crossing of the first derivative.

*For these routines, it is helpful to operate on the smoothed dataset. As we have some side-peaks, we operate on a subrange of the wavelength axis defined by `isig`.*

In [ ]:
peaks = cl2.isig[600.:].find_peaks1D_ohaver(maxpeakn = 1)

The function **returns a structured array** that contains `position`, `height` and `width` for every pixel (potentially each for multiple peaks).

In [ ]:
peaks[0,0]

Especially for broad, asymmetric emission bands, the position of the maximum intensity might be of limited value. Therefore, **LumiSpy** provides an additional `centroid` function that determines the **centre of mass** of a peak.

Required version: lumispy>=0.2.2

*Note that, as with fitting, it might make more sense to run these routines in the energy domain after a Jacobian transformation than to convert the result - in particular for broad emission bands.*

In [ ]:
com = cl2.isig[600.:].centroid()

The result is a new `signal` that we can plot as a colormap using the HyperSpy functionality:

In [ ]:
com.plot(cmap='viridis')

You can also determine the **width of a peak** directly from the signal without fitting a model to the data. Again useful for asymmetric peaks. To plot the FWHM interval, we set `return_interval=True` (the returned list then contains three arrays: *width*, as well as *left position* and *right position* of the interval). 

The default is to determine the **FWHM**, i.e. a `factor=0.5`. This value can be set to any other fraction of the peak height.

In [ ]:
width = cl2.isig[600.:].estimate_peak_width(return_interval=True)

In [ ]:
width[0].plot(cmap='viridis')

Now we can **add markers** for the *FWHM interval* (grey) and the *centre of mass* (black) to the signal object and plot them on the spectra.

*Note that all markers will be renamed in HyperSpy 2.0, `vertical_line` will be replaced by `VerticalLine`*

In [ ]:
mrk = hs.plot.markers.vertical_line(com.data, color='black')
mrkl = hs.plot.markers.vertical_line(width[1].data, color='grey')
mrkr = hs.plot.markers.vertical_line(width[2].data, color='grey')
cl2.add_marker([mrk,mrkl,mrkr], permanent=True)
cl2.plot()

## Axes types / Convert to energy scale

*(Required versions: hyperspy>=1.7.0 and lumispy>=0.2.0)*

HyperSpy has different types of axes:
- The standard `UniformDataAxis` is defined through an `offset` and a `scale` (delta between pixels)
- A `FunctionalDataAxis` is defined through a `UniformDataAxis` and a `function` to convert the values
- A more general `DataAxis` is defined through an `axis` vector/array

The *wavelength* scale of our sample object is a `UniformDataAxis`:

In [ ]:
cl2a.axes_manager

*LumiSpy* provides easy conversions of the signal axis to the **energy scale**:

It can either replace the axis in the existing object (default) or create a copy of the signal object with the new axis (`inplace=False`).

*The conversion routine also performs a Jacobian transformation on the intensity to ensure that integrated signals in certain ranges remain comparable. See the [user guide](https://docs.lumispy.org/en/latest/user_guide/signal_axis.html#jacobian-transformation) for further information.*

In [ ]:
cl2_eV = cl2a.to_eV(inplace=False)

The signal axis is now a *non-uniform axis*:

In [ ]:
cl2_eV.axes_manager

This axis is defined through an axis vector:

In [ ]:
cl2_eV.axes_manager[-1].axis

In [ ]:
cl2_eV.plot()

## Model fitting

We will introduce very basic fitting functionality using the non-smoothed version of our dataset. For more details see other tutorials among the [LumiSpy-demos](https://github.com/LumiSpy/lumispy-demos) and the `Fitting_tutorial` in the [HyperSpy demos repository](https://github.com/hyperspy/hyperspy-demos).

Note that HyperSpy has a range of [built-in functions](https://hyperspy.org/hyperspy-doc/current/user_guide/model.html#pre-defined-model-components) covering most needs that can be added as components to a model. However, it also has an intuitive mechanism to [define custom functions](https://hyperspy.org/hyperspy-doc/current/user_guide/model.html#define-components-from-a-mathematical-expression).

First, we need to **initialize the model**:

In [ ]:
m = cl2_eV.create_model()

**Check the components** of the model (should be empty, but for some types of signals like EDS and EELS, the model is automatically initialized with components):

In [ ]:
m.components

We need to **create some components** and **add them to the model**.

As the emission peak in our dataset is rather asymmetric, we will use a single `SkewNormal` component:

In [ ]:
g1 = hs.model.components1D.SkewNormal()
m.append(g1) # m.extend([g1,g2])
m.components

To see the parameters of our components and their default values, we can **print all parameter values**:

In [ ]:
m.print_current_values()

To apply the fit to all the spectra in the map, we use the `multifit` command.

In the current case of a single, well defined peak, we achieve a good fit without adjusting the initial values of the parameters or setting any boundaries.

In [ ]:
m.multifit()

We can now **plot the model** together with the data:

In [ ]:
m.plot()

The `SkewNormal` component represents the asymmetry of the peak very well, but does not fully reproduce the height of the main part of the peak.

We can also print the parameter values at the current index:

In [ ]:
m.print_current_values()

### Now try with your own data!